Merge multiple videos and export a GIF or MP4 optimized for GitHub / social media.
Designed to convert screen recordings directly into demo videos.
Supported input formats: `.mov` `.mp4` `.avi` `.mkv` `.webm`


##### Choosing an output format

| Format | Characteristics | Best for | Not ideal for |
|---|---|---|---|
| GIF | Auto-plays, loops, works in any Markdown. 256-color limit, larger file size | GitHub README (auto-play required), Zenn / Qiita | Situations where file size is critical |
| MP4 | High quality, small file, full color. Requires a media player to play | X / Slack / GitHub README (click-to-play is fine) | Situations where auto-play or looping is required |


##### How to use — just run cells top to bottom

| Step | Description | Action |
|:---:|---|---|
| ① | Setup | Automatic |
| ② | Upload videos | File selection dialog |
| ③ | Set merge order | Dropdown (skip if only one file) |
| ④ | Configure format, quality & speed | Radio buttons |
| ⑤ | Run conversion | Automatic |
| ⑥ | Preview output | Automatic |
| ⑦ | Choose destination and save | Radio buttons |

> ※ Upload limit (②): ~`2 GB` per file; multiple files: approx. `2 GB × number of files`


##### Reference: Platform size limits (as of Feb 2026)

| Platform | Supported formats | Size limit |
|---|---|---|
| GitHub README | GIF / MP4 / MOV | 100 MB (video) / 10 MB (GIF) |
| Zenn | GIF | 3 MB |
| Qiita | GIF / MP4 | 100 MB |
| X (standard) | MP4 / MOV | 512 MB |
| Slack | GIF / MP4 / MOV and more | 1 GB |

In [ ]:
# ① Setup
!apt-get install -y ffmpeg -q

from google.colab import files as colab_files, drive
import os, shutil, ipywidgets as w
from IPython.display import display, Image

print('✅ Ready')

In [ ]:
# ② Upload (multiple files supported)
print('📂 Select video file(s) (supported formats: .mov .mp4 .avi .mkv .webm)')
uploaded = colab_files.upload()

VIDEO_EXT = ('.mov', '.mp4', '.avi', '.mkv', '.webm')
uploaded_videos = [f for f in uploaded.keys() if f.lower().endswith(VIDEO_EXT)]

if not uploaded_videos:
    print('❌ No video files found')
else:
    print(f'\n✅ {len(uploaded_videos)} file(s) detected:')
    for i, f in enumerate(uploaded_videos):
        size = os.path.getsize(f) / 1024 / 1024
        print(f'   [{i}] {f} ({size:.1f} MB)')

In [ ]:
# ③ Merge order (skip if only one video)
if len(uploaded_videos) == 1:
    order_widgets = None
    print(f'✅ Only one video detected, skipping order selection: {uploaded_videos[0]}')
else:
    print('🔢 Specify the merge order')
    order_widgets = [
        w.Dropdown(options=uploaded_videos, value=uploaded_videos[i],
                   description=f'#{i+1}:', style={'description_width': '60px'},
                   layout=w.Layout(width='420px'))
        for i in range(len(uploaded_videos))
    ]
    display(w.VBox(order_widgets))

In [ ]:
# ④ Output format, quality & speed settings

def make_toggle(widget, show_when):
    def _cb(change):
        widget.layout.display = '' if change['new'] == show_when else 'none'
    return _cb

format_sel = w.RadioButtons(
    options=[
        ('GIF  — Looping playback, great for GitHub README', 'gif'),
        ('MP4 (H.264) — High quality, small size, great for SNS/Slack', 'mp4'),
    ],
    value='gif', description='Format:', style={'description_width': '60px'},
    layout=w.Layout(width='480px')
)

# GIF settings
gif_quality_sel = w.RadioButtons(
    options=[
        ('GitHub README    960px / 15fps', 'github'),
        ('SNS / Lightweight  640px / 10fps', 'sns'),
        ('High Quality      1280px / 20fps', 'hq'),
        ('Custom',                           'custom'),
    ],
    value='github', description='Quality:', style={'description_width': '60px'},
    layout=w.Layout(width='420px')
)
gif_custom_scale  = w.BoundedIntText(value=960,  min=240, max=1920, description='Width px:',      layout=w.Layout(width='220px'), style={'description_width': '80px'})
gif_custom_fps    = w.BoundedIntText(value=15,   min=1,   max=30,   description='FPS:',           layout=w.Layout(width='180px'), style={'description_width': '80px'})
gif_custom_colors = w.BoundedIntText(value=256,  min=16,  max=256,  description='Colors (16-256):', layout=w.Layout(width='240px'), style={'description_width': '110px'})
gif_custom_dither = w.Dropdown(
    options=[('bayer (lightweight)', 'bayer'), ('floyd_steinberg (high quality)', 'floyd_steinberg')],
    value='floyd_steinberg',
    description='Dithering:', layout=w.Layout(width='320px'), style={'description_width': '90px'}
)
gif_custom_box = w.VBox(
    [gif_custom_scale, gif_custom_fps, gif_custom_colors, gif_custom_dither],
    layout=w.Layout(margin='0 0 0 20px', display='none')
)
gif_box = w.VBox([w.HTML('<b>🎨 GIF Quality</b>'), gif_quality_sel, gif_custom_box])
gif_quality_sel.observe(make_toggle(gif_custom_box, 'custom'), names='value')

# MP4 settings
mp4_quality_sel = w.RadioButtons(
    options=[
        ('High quality  (CRF 18) — larger file', 18),
        ('Standard      (CRF 23) — balanced',    23),
        ('Lightweight   (CRF 28) — smaller file', 28),
        ('Custom',                                -1),
    ],
    value=23, description='Quality:', style={'description_width': '60px'},
    layout=w.Layout(width='420px')
)
mp4_custom_crf = w.BoundedIntText(
    value=23, min=0, max=51, description='CRF (0-51):',
    layout=w.Layout(width='220px', display='none'), style={'description_width': '90px'}
)
mp4_scale = w.BoundedIntText(value=960, min=240, max=1920, description='Width px:', layout=w.Layout(width='220px'), style={'description_width': '80px'})
mp4_fps   = w.BoundedIntText(value=30,  min=1,   max=60,   description='FPS:',      layout=w.Layout(width='180px'), style={'description_width': '80px'})
mp4_audio = w.Checkbox(value=False, description='Keep audio', layout=w.Layout(margin='4px 0'))
mp4_box = w.VBox([
    w.HTML('<b>🎨 MP4 Quality</b>'), mp4_quality_sel, mp4_custom_crf,
    w.HTML('<b>📐 Resolution & Frame Rate</b>'), mp4_scale, mp4_fps,
    w.HTML('<b>🔊 Audio</b>'), mp4_audio,
])
mp4_quality_sel.observe(make_toggle(mp4_custom_crf, -1), names='value')
settings_container = w.VBox([gif_box])

def on_format_change(change):
    settings_container.children = [gif_box] if change['new'] == 'gif' else [mp4_box]
format_sel.observe(on_format_change, names='value')

# Playback speed
speed_sel = w.RadioButtons(
    options=[('Normal (1.0x)', 1.0), ('1.5x', 1.5), ('2.0x', 2.0), ('0.75x', 0.75), ('Custom', -1)],
    value=1.0, description='Speed:', style={'description_width': '60px'}
)
custom_speed = w.BoundedFloatText(
    value=1.0, min=0.1, max=10.0, step=0.1,
    description='Multiplier:', layout=w.Layout(width='210px', display='none'), style={'description_width': '80px'}
)
speed_sel.observe(make_toggle(custom_speed, -1), names='value')

display(w.VBox([
    w.HTML('<b>📦 Output Format</b>'), format_sel,
    w.HTML('<hr style="margin:12px 0">'),
    settings_container,
    w.HTML('<hr style="margin:12px 0">'),
    w.HTML('<b>⚡ Playback Speed</b>'), speed_sel, custom_speed,
]))

In [ ]:
# ⑤ Run conversion (re-run only this cell after changing settings)

if order_widgets is None:
    ordered_videos = uploaded_videos
else:
    ordered_videos = [ow.value for ow in order_widgets]
    if len(set(ordered_videos)) != len(ordered_videos):
        dupes = [f for f in set(ordered_videos) if ordered_videos.count(f) > 1]
        raise ValueError(f'❌ Duplicate file(s) detected: {dupes}')

out_format   = format_sel.value
OUTPUT_NAME  = f'demo.{out_format}'
speed        = custom_speed.value if speed_sel.value == -1 else speed_sel.value
speed_filter = f'setpts={1/speed:.4f}*PTS,' if speed != 1.0 else ''

print('📋 Conversion settings')
print(f'   Merge order : {" → ".join(ordered_videos)}')
print(f'   Output format: {out_format.upper()}')
print(f'   Playback speed: {speed}x')

if len(ordered_videos) == 1:
    source = ordered_videos[0]
else:
    with open('list.txt', 'w') as f:
        for v in ordered_videos:
            f.write(f"file '{v}'\n")
    !ffmpeg -y -f concat -safe 0 -i list.txt -c copy combined.mov -loglevel warning
    source = 'combined.mov'
    print(f'✅ Merged {len(ordered_videos)} files')

print('⏳ Converting...')

if out_format == 'gif':
    presets = {
        'github': {'scale': 960,  'fps': 15, 'colors': 256, 'dither': 'floyd_steinberg'},
        'sns':    {'scale': 640,  'fps': 10, 'colors': 128, 'dither': 'bayer'},
        'hq':     {'scale': 1280, 'fps': 20, 'colors': 256, 'dither': 'floyd_steinberg'},
    }
    if gif_quality_sel.value == 'custom':
        scale, fps, colors, dither = gif_custom_scale.value, gif_custom_fps.value, gif_custom_colors.value, gif_custom_dither.value
    else:
        p = presets[gif_quality_sel.value]
        scale, fps, colors, dither = p['scale'], p['fps'], p['colors'], p['dither']
    print(f'   Quality : {scale}px / {fps}fps / {colors} colors / dither={dither}')
    filter_str = (
        f"[0:v] {speed_filter}fps={fps},scale={scale}:-1:flags=lanczos,split [a][b];"
        f"[a] palettegen=max_colors={colors}:stats_mode=full [p];"
        f"[b][p] paletteuse=dither={dither}:diff_mode=rectangle"
    )
    !ffmpeg -y -i "{source}" -filter_complex "{filter_str}" {OUTPUT_NAME} -loglevel warning

elif out_format == 'mp4':
    crf   = mp4_custom_crf.value if mp4_quality_sel.value == -1 else mp4_quality_sel.value
    scale = mp4_scale.value
    fps   = mp4_fps.value
    audio_opt = '' if mp4_audio.value else '-an'
    print(f'   Quality : {scale}px / {fps}fps / CRF={crf} / Audio={"on" if mp4_audio.value else "off"}')
    vf = f"{speed_filter}fps={fps},scale={scale}:-2:flags=lanczos"
    !ffmpeg -y -i "{source}" \
        -vf "{vf}" \
        -c:v libx264 -crf {crf} -preset slow -pix_fmt yuv420p \
        {audio_opt} \
        {OUTPUT_NAME} -loglevel warning

size_mb = os.path.getsize(OUTPUT_NAME) / 1024 / 1024
print(f'\n✅ Done → {OUTPUT_NAME} ({size_mb:.1f} MB)')

if out_format == 'gif' and size_mb > 15:
    print('\n⚠️  File exceeds 15 MB. Try the following in step ④:')
    print('   → Switch quality to "SNS / Lightweight"')
    print('   → Increase playback speed to 1.5x or higher')
    print('   → Or switch the output format to MP4 for a significant size reduction')

In [ ]:
# ⑥ Preview
if out_format == 'gif':
    display(Image(OUTPUT_NAME))
elif out_format == 'mp4':
    from IPython.display import Video
    display(Video(OUTPUT_NAME, embed=True, width=720))

In [ ]:
# ⑦ Choose save destination
save_sel = w.RadioButtons(
    options=[
        ('Download to local machine', 'local'),
        ('Save to Google Drive',      'drive'),
        ('Both',                      'both'),
    ],
    value='local', description='Save to:', style={'description_width': '70px'}
)
drive_path = w.Text(
    value=f'MyDrive/demo.{out_format}',
    description='Drive path:',
    layout=w.Layout(width='380px', display='none'),
    style={'description_width': '90px'}
)

def on_save_change(change):
    drive_path.layout.display = '' if change['new'] in ('drive', 'both') else 'none'
save_sel.observe(on_save_change, names='value')

display(w.VBox([w.HTML('<b>💾 Save Destination</b>'), save_sel, drive_path]))

In [ ]:
# ⑦ Execute save
method = save_sel.value

if method in ('local', 'both'):
    colab_files.download(OUTPUT_NAME)
    print('✅ Downloaded to local machine')

if method in ('drive', 'both'):
    drive.mount('/content/drive')
    dest = f'/content/drive/{drive_path.value}'
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy(OUTPUT_NAME, dest)
    print(f'✅ Saved to Google Drive → {drive_path.value}')